In [17]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from os import walk
from sklearn.feature_selection import mutual_info_regression, f_regression
from sklearn.feature_selection import SelectKBest
from xgboost import XGBRegressor

In [27]:
df_train = pd.read_csv('/Volumes/KM/Archive/KaggleStuff/2017SBER/transformed_data/new/train_v4.csv')
df_test = pd.read_csv('/Volumes/KM/Archive/KaggleStuff/2017SBER/transformed_data/new/test_v4.csv')

In [28]:
df_test.columns

Index(['id', 'full_sq', 'sport_count_5000', 'sport_count_3000',
       'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km',
       'sport_count_2000', 'bulvar_ring_km', 'kremlin_km', 'ttk_km',
       'trc_sqm_5000', 'nuclear_reactor_km', 'sport_count_1500',
       'office_sqm_5000', 'sport_objects_raion', 'trc_count_3000',
       'stadium_km', 'cafe_count_5000_price_1000', 'detention_facility_km',
       'basketball_km', 'cafe_count_5000_price_1500', 'office_km',
       'cafe_count_5000', 'cafe_count_5000_na_price', 'university_km',
       'trc_sqm_3000', 'cafe_count_5000_price_500', 'workplaces_km',
       'cafe_count_5000_price_2500', 'office_sqm_3000', 'swim_pool_km',
       'thermal_power_plant_km', 'office_count_5000', 'catering_km',
       'exhibition_km', 'church_count_5000', 'office_sqm_2000',
       'cafe_count_5000_price_high', 'cafe_count_5000_price_4000',
       'big_church_km', 'school_education_centers_raion', 'sport_count_1000',
       'fitness_km', 'metro_min_avto', 'm

In [29]:
def count_nans(field, dff):
    return (pd.isnull(dff[field]).sum())

In [30]:
count_nans('price', df_train)

0

In [32]:
df_train['full_sq'][df_train['full_sq'] == 0] = df_train['life_sq'][df_train['full_sq'] == 0]
df_test['full_sq'][df_test['full_sq'] == 0] = df_test['life_sq'][df_test['full_sq'] == 0]

df_train['full_sq'][df_train['full_sq'] == 0] = 1

/Users/georgeoblapenko/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/georgeoblapenko/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [34]:
curr = 'usd'
curr = 'eur'
# curr = 'rub'

log_scale = True

if curr == 'rub':
    target = 'price'
else:
    target = 'price_' + curr

features_001 = ['id', 'life_sq', 'max_floor', 'num_room', 'kitch_sq', 'build_year', 'state', 'state_xg1', 'material']
    
features = [c for c in df_test.columns if c not in features_001]
# df_train = df_train[df_train['material_flag'] == False]
features.append('lsq_fsq')

df_train = df_train[~pd.isnull(df_train['floor'])]

df_train['lsq_fsq'] = df_train['life_sq_lg1'] / df_train['full_sq']
df_test['lsq_fsq'] = df_test['life_sq_lg1'] / df_test['full_sq']
df_train['ksq_fsq'] = df_train['kitch_sq_lg1'] / df_train['full_sq']
df_test['ksq_fsq'] = df_test['kitch_sq_lg1'] / df_test['full_sq']



target_scaler = StandardScaler()

if log_scale:
    target_vals = np.log(df_train[target].values + 1)
else:
    target_scaler.fit(df_train[target].values.reshape(-1, 1))
    target_vals = target_scaler.transform(df_train[target].values.reshape(-1, 1))
# target_vals = target_vals.ravel()

train_features = df_train[features].values

# f_k = SelectKBest(f_regression, 50)
# train_features = f_k.fit_transform(df_train[features].values, target_vals)

scaler = StandardScaler()
scaler.fit(train_features)

train_features = scaler.transform(train_features)

In [35]:
print(features)

['full_sq', 'sport_count_5000', 'sport_count_3000', 'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'sport_count_2000', 'bulvar_ring_km', 'kremlin_km', 'ttk_km', 'trc_sqm_5000', 'nuclear_reactor_km', 'sport_count_1500', 'office_sqm_5000', 'sport_objects_raion', 'trc_count_3000', 'stadium_km', 'cafe_count_5000_price_1000', 'detention_facility_km', 'basketball_km', 'cafe_count_5000_price_1500', 'office_km', 'cafe_count_5000', 'cafe_count_5000_na_price', 'university_km', 'trc_sqm_3000', 'cafe_count_5000_price_500', 'workplaces_km', 'cafe_count_5000_price_2500', 'office_sqm_3000', 'swim_pool_km', 'thermal_power_plant_km', 'office_count_5000', 'catering_km', 'exhibition_km', 'church_count_5000', 'office_sqm_2000', 'cafe_count_5000_price_high', 'cafe_count_5000_price_4000', 'big_church_km', 'school_education_centers_raion', 'sport_count_1000', 'fitness_km', 'metro_min_avto', 'market_count_5000', 'park_km', 'big_church_count_5000', 'leisure_count_5000', 'office_sqm_1500', 'ekder_male', 

In [103]:
regressor1 = XGBRegressor(max_depth=2, learning_rate=0.014, n_estimators=2500, subsample=0.78, gamma=0.0) 
regressor2 = GradientBoostingRegressor(max_depth=4, learning_rate=0.012, n_estimators=2000, subsample=0.75)

In [104]:
n_splits = 2
def validate():
    kf = KFold(n_splits=n_splits)
    res = []
    for train_index, test_index in kf.split(train_features):
        regressor2.fit(train_features[train_index], target_vals[train_index])
        pred = regressor2.predict(train_features[test_index])
        
        if log_scale:
            truth = target_vals[test_index]
            res.append(np.sqrt(np.mean(np.square(pred - truth))))
        else:
            pred = target_scaler.inverse_transform(pred)
            truth = target_scaler.inverse_transform(target_vals[test_index])
            res.append(np.sqrt(np.mean(np.square(np.log(pred + 1) - np.log(truth + 1)))))
    return res
r = validate()
print(r, sum(r) / n_splits)

[0.50096265057830425, 0.47550892149506013] 0.488235786037


In [114]:
regressor3 = XGBRegressor(max_depth=3, learning_rate=0.014, n_estimators=2200, subsample=0.75, gamma=0.0) 
regressor3.fit(train_features, target_vals)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.0, learning_rate=0.014, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=2200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.75)

In [105]:
regressor1.fit(train_features, target_vals)
regressor2.fit(train_features, target_vals)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.012, loss='ls', max_depth=4,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=2000, presort='auto', random_state=None,
             subsample=0.75, verbose=0, warm_start=False)

In [115]:
test_features = df_test[features].values
# test_features = f_k.transform(df_test[features].values)
test_features = scaler.transform(test_features)

In [116]:
out1 = regressor1.predict(test_features)
out2 = regressor2.predict(test_features)
out3 = regressor3.predict(test_features)
if log_scale:
    out_scaled1= np.exp(out1) - 1
    out_scaled2= np.exp(out2) - 1
    out_scaled3= np.exp(out3) - 1
else:
    out_scaled1 = target_scaler.inverse_transform(out1)
    out_scaled2 = target_scaler.inverse_transform(out2)
    out_scaled3 = target_scaler.inverse_transform(out3)

In [117]:
if len(out_scaled1.shape) > 1:
    out_scaled1 = out_scaled1[:, 0]
    out_scaled2 = out_scaled2[:, 0]
    out_scaled3 = out_scaled3[:, 0]

In [123]:
w1 = 0.6
w2 = 0.2
w3 = 0.2

dfn_test = pd.DataFrame()

if curr == 'rub':
    dfn_test['price_doc'] = (w1 * out_scaled1 + w2 * out_scaled2 + w3 * out_scaled3)
elif curr == 'usd':
    dfn_test['price_doc'] = df_test['usdrub'] * (w1 * out_scaled1 + w2 * out_scaled2 + w3 * out_scaled3)
else:
    dfn_test['price_doc'] = df_test['eurrub'] * (w1 * out_scaled1 + w2 * out_scaled2 + w3 * out_scaled3)

In [124]:
dfn_test['id'] = df_test['id']

In [125]:
cutoff = 300000
nonpos = (dfn_test['price_doc'] < cutoff).sum()
print(nonpos)
if nonpos > 0:
    avg_pos = np.average(dfn_test['price_doc'][dfn_test['price_doc'] > cutoff].values)
    dfn_test.loc[dfn_test['price_doc'] < 0, 'price_doc'] = avg_pos
nonpos = (dfn_test['price_doc'] < cutoff).sum()
print(nonpos)

0
0


In [126]:
cutoff = 50000000
(dfn_test['price_doc'] > cutoff).sum()

4

In [127]:
maxname = 0
for root, dirs, files in walk('/Volumes/KM/Archive/KaggleStuff/2017SBER/submissions/new'):
    for name in files:
        if name.endswith('.csv'):
            print(name[:-4], int(name[:-4]))
            maxname += 1
print(maxname)
dfn_test.to_csv('/Volumes/KM/Archive/KaggleStuff/2017SBER/submissions/new/' + str(maxname)+ '.csv', index=False)

0 0
1 1
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
2 2
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
3 3
30 30
31 31
4 4
5 5
6 6
7 7
8 8
9 9
32
